In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline
!pip install geocoder
!pip install selenium
!pip install geopy
!pip install pgeocode
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time
!pip install geolocator.geocode
!pip install geopy
from geopy.geocoders import Nominatim
import json

ERROR: Could not find a version that satisfies the requirement geolocator.geocode (from versions: none)
ERROR: No matching distribution found for geolocator.geocode


In [2]:
df = pd.DataFrame(dict(
        AUTHOR_NAME=list('AAABBCCCCDEEFGG'),
        title=      list('zyxwvutsrqponml')
    ))

df2 = pd.DataFrame(dict(
        AUTHOR_NAME=list('AABCCEGG'),
        title      =list('zwvtrpml'),
        CATEGORY   =list('11223344')
    ))

In [3]:
df

,AUTHOR_NAME,title
0,A,z
1,A,y
2,A,x
3,B,w
4,B,v
5,C,u
6,C,t
7,C,s
8,C,r
9,D,q


In [4]:
df2

,AUTHOR_NAME,title,CATEGORY
0,A,z,1
1,A,w,1
2,B,v,2
3,C,t,2
4,C,r,3
5,E,p,3
6,G,m,4
7,G,l,4


In [5]:
df.merge(df2, how='left')

,AUTHOR_NAME,title,CATEGORY
0,A,z,1
1,A,y,NaN
2,A,x,NaN
3,B,w,NaN
4,B,v,2
5,C,u,NaN
6,C,t,2
7,C,s,NaN
8,C,r,3
9,D,q,NaN


In [50]:
response = requests.get('https://www.torontorentals.com/toronto')
soap = BeautifulSoup(response.text,"html.parser")
soap1 = soap.find_all('script')

soap1 = soap1[-1]
soap1.text

import re
f1 = re.sub("\n    \n      \n        \n        \n        \n\n        ",'', soap1.text)
f1 = re.sub("\n    const markers = \[\];","",f1)
f1 = re.sub("\n        ","",f1)
f1 = re.sub("\n       ",'',f1)
f1 = re.sub('   ','',f1)
f1 = re.sub('\n\n','',f1)
f1 = re.sub('markers.push\(','',f1)
f1 = re.sub('\n','',f1)
f1 = re.sub('\)','',f1)
f2 = re.split(r'\;',f1)


a = f2[52]

print(len(f2))
streetname = re.sub("\'","",re.sub("\"","",re.findall(r'street:\s(.*?),',a)[0]))
name = re.sub("\'","",re.sub("\"","",re.findall(r'name:\s(.*?),',a)[0]))
lat = re.sub("\'","",re.sub("\"","",re.findall(r'lat: (.*?),',a)[0]))
lng = re.sub("(.*?)\,lng: ","",re.findall(r'lat: (.*?)\}',a)[0])

print(streetname)
print(name)
print(lat)
print(lng)

2218
7 Hilts Dr

43.9057877
-79.3929427


"\n    const markers = [];\n    \n      \n        \n        \n        \n\n        markers.push({\n        id: '426',\n        name: 'Bayview Towers',\n        street: '505 Cummer Ave',\n        visible_street: '505 Cummer Ave',\n        type: 'Apartment',\n        rent: [1647.00, 2227.00],\n       url: '/north-york/bayview-towers-id426',\n       city: {\n           'name': 'North York'\n       },\n       location: {\n           lat: 43.792349,\n           lng: -79.393537\n       }\n   });\n      \n        \n        \n        \n\n        markers.push({\n        id: '60237',\n        name: 'Kings Club',\n        street: '1100 King Street West',\n        visible_street: '1100 King Street West',\n        type: 'apartment',\n        rent: [2150.00, 3815.00],\n       url: '/toronto/kings-club-id60237',\n       city: {\n           'name': 'Toronto'\n       },\n       location: {\n           lat: 43.63985,\n           lng: -79.42373\n       }\n   });\n      \n        \n        \n        \n\n  

In [51]:
from geopy.geocoders import Bing
new_bing = Bing(api_key='AutK2PfFsISHtjUQr00rl2Kf_5tlpgYPtZzJUBZnwl8_NyIOydqyWW91RS4N7NQQ')

In [62]:
location = new_bing.(query='60 Ruddington Drive, Toronto',include_neighborhood=0)
location.address

'60 Ruddington Dr, Toronto, ON M2K 2J9, Canada'

In [66]:
url = 'http://dev.virtualearth.net/REST/v1/Locations/CA/60%20Ruddington%20Drive?includeNeighborhood=1&key=AutK2PfFsISHtjUQr00rl2Kf_5tlpgYPtZzJUBZnwl8_NyIOydqyWW91RS4N7NQQ'
response = requests.get(url)
response = response.json()
neighborhood = response['resourceSets'][0]['resources'][0]['address']['neighborhood']
latitude = response['resourceSets'][0]['resources'][0]['point']['coordinates'][0]
longitude = response['resourceSets'][0]['resources'][0]['point']['coordinates'][1]

In [60]:
url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}'

request = url.format('43.79163,-79.39042')
data = requests.get(request).json()
data

{'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account',
 'results': [],
 'status': 'REQUEST_DENIED'}

In [14]:
print(len(f2))
from ast import literal_eval

2218


In [15]:
a = re.sub("^\"*\"$", "",f2[2])

In [16]:
e = re.sub('\'','"',repr(a))
e = re.sub(",rent:\s(.*?)\},location:\s\{(.*?)\}","",e)
#e = re.sub("rent:\s","",e)
#e = re.sub("\[(.*?)\],","",e)
print(e)

re.sub("\"","",re.findall(r'street:\s(.*?),',e)[0])

"{id: "447",name: "70 Ruddington Drive",street: "70 Ruddington Drive",visible_street: "70 Ruddington Drive",type: "Apartment"}"


'70 Ruddington Drive'

In [17]:
#literal_eval(e)
import ast
x = ast.literal_eval(output)
#json.loads(e)
#re.sub('\"','', json.loads(e))
#ast.literal_eval(json.dumps(e, ensure_ascii=False).encode('utf8'))

NameError: name 'output' is not defined

In [ ]:
dict(x)

In [ ]:
b = (re.sub('\}','',re.sub('\{','',a)))

In [ ]:
b

In [ ]:
geolocator = Nominatim(user_agent='my_user')
location = geolocator.geocode('Bayview Towers, CA')
latitude = location.latitude
longitude = location.longitude
print(latitude)
print(longitude)

In [ ]:
dict1